In [1]:
from utils.data_preprocess_utils import get_config
from easydict import EasyDict
from utils.dataset import DenmarkDataset

In [2]:
cfg = EasyDict(get_config('./cfg/dataset_cfg.yaml'))
dataset = DenmarkDataset(cfg)

day_data = dataset._get_day_data('2019-01-01')
range_data = dataset.get_data_by_range('2019-01-01', '2019-03-10')

In [3]:
print(range_data.keys())
total_VTF = day_data['Total']
print(total_VTF.shape)

dict_keys(['Sailing', 'Pleasure', 'Cargo', 'Fishing', 'Passenger', 'Tanker', 'Tug', 'Other', 'Total'])
(4, 103, 4)


In [48]:
import geopandas as gpd
import numpy as np
import pandas as pd
gdf = gpd.read_file("/root/GIS/waypoints/convex_hull.geojson") # waypoint hulls
gdf['label'] = gdf.reset_index().index
gdf['centroid'] = gdf['geometry'].centroid.to_crs(crs='EPSG:4326')
N = gdf['centroid'].shape[0]

df_total = pd.DataFrame()
data = np.load('/root/GIS/ais_flow/2019-02-03.npy',allow_pickle=True).item()
data_total = data['Total']

for i in range(N+1):
        df_total = df_total.copy()
        df_total[i] = [data_total[v0][i] for v0 in range(4)]
        # df_total = df_total.rename(index=dict(zip(df_total.index, idx)))

df_total

/tmp/ipykernel_2836/3631527771.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid.to_crs(crs='EPSG:4326')


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,"[750.0, 516.0, 522.0, 0.043401347415609984]","[1.0, 1.0, 0.0, 18.15]","[24.0, 12.0, 7.0, 0.8568124510413161]","[29.0, 22.0, 19.0, 0.538962663692589]","[0.0, 0.0, 0.0, 0.0]","[20.0, 16.0, 18.0, 1.625937177499134]","[0.0, 0.0, 0.0, 0.0]","[14.0, 13.0, 12.0, 3.366241854484439]","[28.0, 51.0, 50.0, 2.2466104532856215]","[3.0, 6.0, 5.0, 7.075207168057219]",...,"[25.0, 25.0, 23.0, 2.6807754837419635]","[43.0, 16.0, 24.0, 1.559196485075161]","[17.0, 14.0, 14.0, 1.9910372667685219]","[1.0, 0.0, 0.0, 0.0]","[23.0, 17.0, 17.0, 1.6924883545746818]","[22.0, 18.0, 20.0, 1.9818083339706618]","[0.0, 0.0, 0.0, 0.0]","[2.0, 1.0, 1.0, 4.474532710280374]","[2.0, 2.0, 2.0, 2.622775800711744]","[1.0, 1.0, 1.0, 6.622857142857143]"
1,"[770.0, 585.0, 565.0, 0.04650862265889043]","[2.0, 2.0, 1.0, 19.299999999999997]","[27.0, 6.0, 17.0, 0.7763831463124452]","[37.0, 30.0, 32.0, 0.946181920311032]","[0.0, 0.0, 0.0, 0.0]","[35.0, 37.0, 34.0, 1.5166955836487335]","[1.0, 1.0, 1.0, 16.983271375464685]","[28.0, 26.0, 22.0, 1.7968165338374733]","[33.0, 58.0, 57.0, 1.5488201095293181]","[8.0, 16.0, 15.0, 4.393523097794183]",...,"[28.0, 26.0, 27.0, 2.077729545812733]","[43.0, 15.0, 27.0, 1.4612089960829726]","[20.0, 20.0, 19.0, 2.234461923860737]","[4.0, 6.0, 6.0, 6.659278234190375]","[22.0, 18.0, 16.0, 2.0639898069703544]","[18.0, 18.0, 16.0, 3.6403688499100273]","[1.0, 1.0, 0.0, 1.028]","[5.0, 0.0, 4.0, 9.466844380154049]","[5.0, 4.0, 2.0, 5.797676459697824]","[2.0, 2.0, 1.0, 5.638263542066846]"
2,"[785.0, 599.0, 607.0, 0.044504833369771936]","[2.0, 1.0, 2.0, 8.262992125984253]","[20.0, 8.0, 12.0, 0.7781797927701887]","[28.0, 23.0, 24.0, 0.6591970319764456]","[0.0, 0.0, 0.0, 0.0]","[30.0, 26.0, 29.0, 1.5813412245581284]","[0.0, 0.0, 0.0, 0.0]","[25.0, 19.0, 23.0, 2.325020076458337]","[45.0, 67.0, 68.0, 1.2522348627118296]","[11.0, 20.0, 21.0, 4.065040499428604]",...,"[18.0, 17.0, 15.0, 3.905279604991856]","[25.0, 6.0, 18.0, 2.5641065636844234]","[31.0, 30.0, 27.0, 2.0446141650742566]","[3.0, 5.0, 3.0, 2.679148130395791]","[32.0, 25.0, 24.0, 2.041393468922325]","[25.0, 21.0, 22.0, 2.7403495199454513]","[0.0, 0.0, 0.0, 0.0]","[3.0, 3.0, 2.0, 7.868679606997364]","[7.0, 4.0, 4.0, 4.500556554902955]","[4.0, 2.0, 3.0, 6.452106734905801]"
3,"[773.0, 574.0, 571.0, 0.04007874039519133]","[2.0, 1.0, 2.0, 16.295238095238098]","[8.0, 4.0, 4.0, 1.9804497628280529]","[26.0, 21.0, 16.0, 0.6459801988094228]","[0.0, 0.0, 0.0, 0.0]","[23.0, 22.0, 23.0, 1.7668957559188367]","[0.0, 0.0, 0.0, 0.0]","[24.0, 22.0, 21.0, 1.7730094582249423]","[41.0, 68.0, 67.0, 1.4453581466931773]","[8.0, 16.0, 15.0, 5.159133377309923]",...,"[21.0, 18.0, 18.0, 2.194399872986834]","[26.0, 13.0, 13.0, 1.563557865333831]","[28.0, 22.0, 26.0, 1.8072154664842193]","[4.0, 2.0, 3.0, 1.0392857142857144]","[32.0, 24.0, 25.0, 1.2006364447875248]","[26.0, 23.0, 25.0, 1.91920836971283]","[0.0, 0.0, 0.0, 0.0]","[2.0, 1.0, 1.0, 12.947551224387807]","[5.0, 2.0, 4.0, 7.357130353285042]","[2.0, 2.0, 0.0, 3.353]"


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf

time_series_node1_flow = total_VTF[:, 3, 1]
acf_values = acf(time_series_node1_flow, nlags=50)
pacf_values = pacf(time_series_node1_flow, nlags=50)
plt.figure(figsize=(14, 5))
plt.subplot(121)
plt.stem(acf_values)
plt.title('Autocorrelation Function (ACF)')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.subplot(122)
plt.stem(pacf_values)
plt.title('Partial Autocorrelation Function (PACF)')
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorly as tl
from tensorly.decomposition import non_negative_parafac
from scipy.stats import pearsonr

np.random.seed(42)
tensor_data = total_VTF
flow_in = tensor_data[:, :, 1]

rank = 5  
weights, factors = non_negative_parafac(tl.tensor(flow_in), rank=rank, n_iter_max=200)
factor_time, factor_nodes = factors

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(factor_time)
plt.title('Factor: Time')

plt.subplot(1, 2, 2)
plt.plot(factor_nodes)
plt.title('Factor: Nodes')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
print(factor_nodes.shape)
dim1 = factor_nodes[:, 1]
k = 10
topk_indices = np.argsort(dim1)[-k:]
gdf = gpd.read_file('/root/autodl-tmp/waypoints/convex_hull.geojson')
gdf['label'] = gdf.reset_index().index
gdf['centroid'] = gdf['geometry'].centroid.to_crs(crs='EPSG:4326')

m = folium.Map(location=[55.6761, 12.5683], zoom_start=10)
for index in topk_indices:
    if index == 0:
        continue
    index = index+1
    centroid = gdf[gdf['label'] == index]['centroid']
    folium.CircleMarker([centroid.y, centroid.x]).add_to(m)
m